# Initial Parameter

In [1]:
# Set this parameter based on where you save the files

project_path = '/content/drive/My Drive/Shared/Shahriyar Noei'

# Colab

In [ ]:
! pip install pytorch_lightning
! pip install alexnet_pytorch
! pip install humanfriendly

In [ ]:
from google.colab import drive
mounted_dir = '/content/drive/'
drive.mount(mounted_dir, force_remount=True)

In [4]:
import sys
sys.path.append(project_path)

# CORnet-Z

## Train Model

In [ ]:
import os
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from functools import reduce
from cornet.extend_cornet_z import ExtendedCORnet

seed = 0
pl.seed_everything(seed)

# -------------------------------
# Parameters
# -------------------------------

# ESOS: 5
# Testolin: 10
n_max = 5 # max number of dots

# -------------------------------
# Paths where to load/save data
# -------------------------------

# path where model are saved
model_path = project_path+'/result/model/cornetz_sgd'
os.makedirs(f'{model_path}', exist_ok=True)
# path where log of training are saved
log_path = project_path+'/result/log/train'
os.makedirs(f'{log_path}', exist_ok=True)
# path containing the dataset
train_path = project_path+'/images/DeWind/train'

# -------------------------------
# Training dataset
# -------------------------------

# Write transform for image
data_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(300, 300)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0
])

train_data = datasets.ImageFolder(root=train_path, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)
train_loader = DataLoader(dataset=train_data, batch_size = 32, shuffle = True, num_workers = 4, pin_memory = True)

# -------------------------------
# Initializing model
# -------------------------------

model  = ExtendedCORnet(out_features = n_max, lr = 1e-3, optimizer = 'sgd')

# -------------------------------
# Saving model
# -------------------------------

# saving initial model
torch.save({
	"epoch": -1,
	"global_step": 0,
	"pytorch-lightning_version": pl.__version__,
	"state_dict": model.state_dict()
}, f'{model_path}/epoch-1.ckpt')
# using checkpoint to save models after each epoch
checkpoint = pl.callbacks.ModelCheckpoint(dirpath = model_path, filename = 'epoch{epoch:02d}', auto_insert_metric_name = False, save_on_train_epoch_end = True, save_top_k = -1)
# saving gpu stats
gpu_stats = pl.callbacks.DeviceStatsMonitor()

# -------------------------------
# Training model
# -------------------------------

trainer = pl.Trainer(default_root_dir = log_path, callbacks = [gpu_stats, checkpoint], deterministic = True, accelerator = 'gpu', devices = 1, strategy = 'auto', num_nodes = 1, max_epochs = 100)
#trainer = pl.Trainer(default_root_dir = log_path, callbacks = [gpu_stats, checkpoint], deterministic = True, accelerator = 'gpu', devices = 4, strategy = 'dpp', num_nodes = 1, max_epochs = 100)
trainer.fit(model, train_loader)

## Test Model

In [ ]:
import os
import zipfile
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pytorch_lightning as pl
from functools import reduce
from cornet.extend_cornet_z import ExtendedCORnet

seed = 0
pl.seed_everything(seed)

# -------------------------------
# Parameters
# -------------------------------

# ESOS: 5
# Testolin: 10
n_max = 5 # max number of dots

# -------------------------------
# Paths where to load/save data
# -------------------------------

# path containing the dataset
test_path = project_path+'/images/DeWind/test'
# path containing the model
model_path = project_path+'/result/model/cornetz_sgd'

# -------------------------------
# Test dataset
# -------------------------------

# Write transform for image
data_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(300, 300)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0
])

#test_data = ZipDataset(test_path, transform=data_transform)
test_data = datasets.ImageFolder(root=test_path, transform=data_transform)
test_loader = DataLoader(dataset=test_data, batch_size = 32, shuffle = False, num_workers = 4, pin_memory = True)

# -------------------------------
# Initializing model
# -------------------------------

model  = ExtendedCORnet(out_features = n_max)

checkpoint = torch.load(f'{model_path}/cornetz.ckpt')
model.load_state_dict(checkpoint['state_dict'])

trainer = pl.Trainer(deterministic=True, devices="auto", accelerator="auto")
metrics, = trainer.test(model, test_loader)

# CORnet-S

## Train Model

In [ ]:
import os
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from functools import reduce
from cornet.extend_cornet_s import ExtendedCORnet

seed = 0
pl.seed_everything(seed)

# -------------------------------
# Parameters
# -------------------------------

# ESOS: 5
# Testolin: 10
n_max = 10 # max number of dots

# -------------------------------
# Paths where to load/save data
# -------------------------------

# path where model are saved
model_path = project_path+'/result/model/cornets_sgd'
os.makedirs(f'{model_path}', exist_ok=True)
# path where log of training are saved
log_path = project_path+'/result/log/train'
os.makedirs(f'{log_path}', exist_ok=True)
# path containing the dataset
train_path = project_path+'/images/DeWind/train'

# -------------------------------
# Training dataset
# -------------------------------

# Write transform for image
data_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(300, 300)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0
])

train_data = datasets.ImageFolder(root=train_path, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)
train_loader = DataLoader(dataset=train_data, batch_size = 32, shuffle = True, num_workers = 4, pin_memory = True)

# -------------------------------
# Initializing model
# -------------------------------

model  = ExtendedCORnet(out_features = n_max, lr = 1e-3, optimizer = 'sgd')

# -------------------------------
# Saving model
# -------------------------------

# saving initial model
torch.save({
	"epoch": -1,
	"global_step": 0,
	"pytorch-lightning_version": pl.__version__,
	"state_dict": model.state_dict()
}, f'{model_path}/epoch-1.ckpt')
# using checkpoint to save models after each epoch
checkpoint = pl.callbacks.ModelCheckpoint(dirpath = model_path, filename = 'epoch{epoch:02d}', auto_insert_metric_name = False, save_on_train_epoch_end = True, save_top_k = -1)
# saving gpu stats
gpu_stats = pl.callbacks.DeviceStatsMonitor()

# -------------------------------
# Training model
# -------------------------------

trainer = pl.Trainer(default_root_dir = log_path, callbacks = [gpu_stats, checkpoint], deterministic = True, accelerator = 'gpu', devices = 1, strategy = "auto", num_nodes = 1, max_epochs = 100)
#trainer = pl.Trainer(default_root_dir = log_path, callbacks = [gpu_stats, checkpoint], deterministic = True, accelerator = 'gpu', devices = 4, strategy = 'dpp', num_nodes = 1, max_epochs = 100)
trainer.fit(model, train_loader)

## Test Model

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pytorch_lightning as pl
from functools import reduce
from cornet.extend_cornet_s import ExtendedCORnet

seed = 0
pl.seed_everything(seed)

# -------------------------------
# Parameters
# -------------------------------

# ESOS: 5
# Testolin: 10
n_max = 10 # max number of dots

# -------------------------------
# Paths where to load/save data
# -------------------------------

# path containing the dataset
test_path = project_path+'/images/DeWind/test'
# path containing the model
model_path = project_path+'/result/model/cornets_sgd'

# -------------------------------
# Test dataset
# -------------------------------

# Write transform for image
data_transform = transforms.Compose([
    # Resize the images to 64x64
    #transforms.Resize(size=(64, 64)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0
])

test_data = datasets.ImageFolder(root=test_path, transform=data_transform)
test_loader = DataLoader(dataset=test_data, batch_size = 32, shuffle = False, num_workers = 4, pin_memory = True)

# -------------------------------
# Initializing model
# -------------------------------

model  = ExtendedCORnet(out_features = n_max)

checkpoint = torch.load(f'{model_path}/cornets.ckpt')
model.load_state_dict(checkpoint['state_dict'])

trainer = pl.Trainer(deterministic=True, devices="auto", accelerator="auto")
metrics, = trainer.test(model, test_loader)